So I could choose to use the inbuilt `compile` method. Alternatively I could pursue my own fun and form of quaility.

In [88]:
from h5py import File
from os import getcwd

from numpy import array, diag, transpose   # Array manipulation 
from numpy import log, sum, exp   # Mathematical operations
from numpy import float32 # Types

In [99]:
model = File(f"{getcwd()}/models/Guttler14.hd5")
model = {key: array([*value]) for key, value in model.items()}

In [100]:
decay = diag([log(2) / 5730] * len(model["fluxes"]))
fluxes = model["fluxes"] / transpose(model["reservoir content"])
balance = diag(sum(fluxes, axis=1))
transfer = transpose(fluxes) - balance - decay
projection = model["production coefficients"]

In [ ]:
def production(t, *args):
    start, duration, phase, area = args

    middle = start + duration / 2.
    height = area / duration
    gaussian = height * exp(- ((t - middle) / (duration / 2)) ** 16.)
    
    production = steady_state_production + 0.18 * steady_state_production * sin(
        2 * pi / 11 * t + phase) + height
    return production

In [97]:
def dydx(y, t, *args):
    return transfer @ y + production(t, *args) * projection